# 02 嵌入生成与向量存储

本 Notebook 为清洗后的产品和评论数据生成嵌入向量，并写入 LanceDB。

**前置要求**：
- 已运行 Notebook 01（生成了 `data/products_clean.parquet` 和 `data/reviews_clean.parquet`）
- 已配置 SiliconFlow API Key

## 1. 加载清洗数据

读取 Notebook 01 输出的 Parquet 文件。

In [ ]:
import daft
from daft import col

df_products = daft.read_parquet("../data/products_clean.parquet")
df_reviews = daft.read_parquet("../data/reviews_clean.parquet")

print(f"产品数据: {df_products.count_rows()} 条")
print(f"评论数据: {df_reviews.count_rows()} 条")

## 2. 配置嵌入 Provider

Daft 内置了 `embed_text` 函数，通过 `set_provider` 配置 API 提供商后，可以直接在 DataFrame 上批量生成嵌入。

相比 Demo 3 中手动调用 OpenAI SDK 分批处理，Daft 的方式更简洁：
- 自动处理批量和并发
- 支持 lazy evaluation（嵌入生成作为 pipeline 的一部分延迟执行）
- 切换 Ray Runner 即可分布式加速

In [ ]:
import os

assert os.environ.get("OPENAI_API_KEY"), "请设置环境变量 OPENAI_API_KEY"

daft.set_provider(
    "openai",
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1"),
)

EMBEDDING_MODEL = "Qwen/Qwen3-Embedding-4B"
EMBEDDING_DIM = 1024

print(f"Provider: {daft.current_provider()}")
print(f"Model: {EMBEDDING_MODEL} ({EMBEDDING_DIM}d)")

## 3. 产品嵌入生成

用 `embed_text` 为产品的 `search_text` 列生成嵌入向量。`search_text` 包含了品牌、子类别和描述信息。

In [ ]:
from daft.functions.ai import embed_text

df_products_embedded = df_products.with_column(
    "vector",
    embed_text(
        col("search_text"),
        model=EMBEDDING_MODEL,
        dimensions=EMBEDDING_DIM,
    ),
)

# 触发计算并转为 pandas
print("正在生成产品嵌入...")
products_pd = df_products_embedded.to_pandas()
print(f"完成: {len(products_pd)} 条产品，向量维度 {len(products_pd['vector'].iloc[0])}")

## 4. 评论嵌入生成

同样的方式为评论的 `review` 列生成嵌入。

In [ ]:
df_reviews_embedded = df_reviews.with_column(
    "vector",
    embed_text(
        col("review"),
        model=EMBEDDING_MODEL,
        dimensions=EMBEDDING_DIM,
    ),
)

print("正在生成评论嵌入...")
reviews_pd = df_reviews_embedded.to_pandas()
print(f"完成: {len(reviews_pd)} 条评论，向量维度 {len(reviews_pd['vector'].iloc[0])}")

## 5. 写入 LanceDB

将产品和评论分别写入 LanceDB 的两张表。回顾 Demo 3 的知识：LanceDB 是嵌入式向量数据库，数据存储在本地 Lance 格式文件中。

In [ ]:
import lancedb

db = lancedb.connect("../lancedb_data")

# 写入产品表
products_table = db.create_table(
    "products", products_pd.to_dict("list"), mode="overwrite"
)
print(f"产品表: {products_table.count_rows()} 条")

# 写入评论表
reviews_table = db.create_table(
    "reviews", reviews_pd.to_dict("list"), mode="overwrite"
)
print(f"评论表: {reviews_table.count_rows()} 条")

print(f"\n数据库表: {db.table_names()}")

## 6. 创建索引

为两张表创建 IVF-PQ 索引，加速后续的向量搜索。

> 注意：小数据集（< 5000 条）索引加速效果不明显，但这里演示完整流程。

In [ ]:
# 产品表索引
products_table.create_index(
    metric="L2",
    num_partitions=8,
    num_sub_vectors=32,
)
print("产品表索引创建完成")

# 评论表索引
reviews_table.create_index(
    metric="L2",
    num_partitions=16,
    num_sub_vectors=32,
)
print("评论表索引创建完成")

## 7. 快速验证

简单测试一下搜索是否正常工作。

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1"),
)


def get_query_vector(query: str) -> list[float]:
    """生成查询文本的嵌入向量"""
    response = client.embeddings.create(input=[query], model=EMBEDDING_MODEL)
    return response.data[0].embedding


# 测试产品搜索
query_vec = get_query_vector("拍照好的手机")
results = products_table.search(query_vec).limit(3).to_pandas()
print("产品搜索: '拍照好的手机'")
for _, row in results.iterrows():
    print(f"  [{row['category']}/{row['subcategory']}] {row['name']} - ¥{row['price']}")

In [ ]:
# 测试评论搜索
query_vec = get_query_vector("质量好值得购买")
results = reviews_table.search(query_vec).limit(3).to_pandas()
print("评论搜索: '质量好值得购买'")
for _, row in results.iterrows():
    print(f"  [{row['cat']}] (label={row['label']}) {row['review'][:60]}")

## 8. 扩展：Ray Runner 加速

当数据量从千级增长到百万级时，嵌入生成会成为瓶颈。Daft 支持切换到 Ray Runner 进行分布式执行：

```python
# 只需一行代码切换到 Ray 分布式执行（回顾 Demo 2）
daft.context.set_runner_ray()

# 同样的 pipeline 代码，自动分布式执行
df_products_embedded = df_products.with_column(
    "vector",
    embed_text(col("search_text"), model=EMBEDDING_MODEL, dimensions=EMBEDDING_DIM),
)
```

Ray Runner 会自动将数据分片到多个 worker 并行调用嵌入 API，显著提升吞吐量。详见 [Demo 2: Ray 分布式计算](../../demo2_ray/)。

## 小结

本 Notebook 完成了嵌入生成与向量存储：

- 用 Daft `embed_text` 为产品和评论批量生成嵌入（回顾 Demo 3 的嵌入概念）
- 写入 LanceDB 的 `products` 和 `reviews` 两张表（回顾 Demo 3 的 LanceDB 操作）
- 创建 IVF-PQ 索引加速搜索
- 提及 Ray Runner 可分布式加速（衔接 Demo 2）

下一步，Notebook 03 将在这些数据上实现智能搜索与推荐功能。